In [2]:
from pathlib import Path
import sys

home_path = str(Path.cwd().parent)
sys.path.append(home_path)

In [6]:
from aper_package.interactive_tool import InteractiveTool

In [7]:
tool = InteractiveTool()

In [8]:
tool.show()

In [7]:
import ipywidgets as widgets
from IPython.display import display

# Placeholder file chooser widgets (replace with your actual widgets)
line_chooser_with_label = widgets.HBox([
    widgets.Label(value="Line File:"),
    widgets.FileUpload()
])

aperture_chooser_with_label = widgets.HBox([
    widgets.Label(value="Aperture File:"),
    widgets.FileUpload()
])

optics_chooser_with_label = widgets.HBox([
    widgets.Label(value="Optics File:"),
    widgets.FileUpload()
])

# Help button setup
help_icon = widgets.Button(
    description="ℹ",
    tooltip="Click for help",
    layout=widgets.Layout(width="30px", height="30px")
)

# Help instructions (adjust content as needed)
instructions = (
    "<b>Line File:</b><br>"
    "1. Select a JSON file from the xsuite folder (e.g. <code>LHC_run3/machine_configurations/2023_MD/xsuite</code>).<br>"
    "   - Choose a file for Beam 1 (e.g. <code>flat_top_b1.json</code>). The path for Beam 2 will be auto-generated.<br>"
    "   - This is the only required file.<br>"
    "2. To modify knobs, use a JSON file with deferred expressions.<br><br>"

    "<b>Aperture File:</b><br>"
    "1. Select a TFS file from the MADX folder (e.g. <code>LHC_run3/machine_configurations/2023_MD/MADX/injection</code>).<br>"
    "   - Choose a file for Beam 1 (e.g., <code>all_optics_B1.tfs</code>). The path for Beam 2 will be auto-generated.<br>"
    "2. If you do not select this file, the aperture will not be visualized.<br><br>"

    "<b>Optics File:</b><br>"
    "1. Select a TFS file from the MADX_thick folder (e.g. <code>LHC_run3/machine_configurations/2023_MD/MADX_thick/injection</code>).<br>"
    "2. If you do not select this file, the optics will not be visualized above the graph.<br><br>"

    "<b>After Selecting All Files:</b><br>"
    "- You can now adjust some graph properties and press the <b>'Apply Changes'</b> button.<br>"
    "- Progress will be displayed just above the graph."
)

help_output = widgets.Output()

def toggle_help(change):
    with help_output:
        help_output.clear_output()  # Clear any previous output
        if help_output.layout.display == "none":
            # Display instructions inside a box below the help button
            display(widgets.HTML(instructions))
            help_output.layout.display = "block"
        else:
            help_output.layout.display = "none"

help_output.layout.display = "none"
help_icon.on_click(toggle_help)

# Group together file choosers
file_chooser_controls = [
    line_chooser_with_label, 
    aperture_chooser_with_label, 
    optics_chooser_with_label,
    help_icon,  # Add help button here
]

# Create layout for the first row of controls (file chooser)
file_chooser_layout = widgets.HBox(
    file_chooser_controls,
    layout=widgets.Layout(
        justify_content="space-between",  # Adjust spacing
        align_items="center",  # Align items properly
        width="100%",
        padding="10px",
        border="solid 1px #ddd",  # Lighter border
    ),
)

# Larger HBox to wrap file_chooser_layout (you can add more widgets to this)
file_chooser_layout

# Larger VBox to hold everything
main_vbox = widgets.VBox([
    larger_hbox,  # Put your HBox inside the VBox
    widgets.Label("Additional Controls Below")  # Another label or control
])

# Display the whole layout including the help instructions box
display(widgets.VBox([main_vbox, help_output]))



In [18]:
help_icon = widgets.Button(
    description="❓",
    tooltip="Click for help"
)


help_icon

Button(description='❓', style=ButtonStyle(), tooltip='Click for help')